# WIP
## To Do
- Experiment with different cross validation techniques from https://github.com/MaartenGr/validation/blob/master/Validation.ipynb
- Test model using tests - e.g. https://towardsdatascience.com/validating-your-machine-learning-model-25b4c8643fb7
- hyperparameter tuning -  https://neptune.ai/blog/hyperparameter-tuning-in-python-complete-guide
- regularisation

In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier

In [10]:
def load_data(bin=False):
    project_dir = os.path.dirname(os.path.abspath(''))
    df = pd.read_json(os.path.join(project_dir, 'model_prepped_dataset.json'))
    X = df.loc[:, ~df.columns.isin(['Outcome', 'Outcome_Bin_H'])]
    if bin:
        y = df['Outcome_Bin_H']
    else:
        y = df['Outcome']
    return X, y
X, y = load_data(bin=True)

def prep_datasets(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train)
    X_test = ss.transform(X_test)
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = prep_datasets(X, y)

In [ ]:
sklearn.ensemble.HistGradientBoostingClassifier
sklearn.ensemble.VotingClassifier
sklearn.ensemble.StackingClassifier
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    RandomForestClassifier()

In [11]:
np.random.seed(2)

models = [RandomForestClassifier()
]

for model in models:
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    accu = accuracy_score(y_test, y_pred) * 100

    print(
        f"{model.__class__.__name__}: "
        f"Accuracy: {accu:.2f}"
    )

RandomForestClassifier: Accuracy: 59.91


In [12]:
model.feature_importances_

array([0.05841513, 0.09390258, 0.08887465, 0.09307559, 0.03405145,
       0.02132811, 0.02396516, 0.02910338, 0.01598044, 0.04740941,
       0.04507409, 0.04524827, 0.04585539, 0.06715538, 0.06589328,
       0.06718503, 0.06499247, 0.09249018])